[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/mongodb-developer/GenAI-Showcase/blob/main/notebooks/evals/angle-embeddings-eval.ipynb)

[![View Article](https://img.shields.io/badge/View%20Article-blue)](https://www.mongodb.com/developer/products/atlas/choose-embedding-model-rag/?utm_campaign=devrel&utm_source=cross-post&utm_medium=organic_social&utm_content=https%3A%2F%2Fgithub.com%2Fmongodb-developer%2FGenAI-Showcase&utm_term=apoorva.joshi)


# RAG Series Part 1: How to choose the right embedding model for your RAG application

This notebook evaluates the [UAE-Large-V1](https://huggingface.co/WhereIsAI/UAE-Large-V1) model.


## Step 1: Install required libraries

- **datasets**: Python library to get access to datasets available on Hugging Face Hub
<p>
- **sentence-transformers**: Framework for working with text and image embeddings
<p>
- **transformers**: Python library that provides APIs to interact with pre-trained models available on Hugging Face
<p>
- **numpy**: Python library that provides tools to perform mathematical operations on arrays
<p>
- **pandas**: Python library for data analysis, exploration and manipulation
<p>
- **tdqm**: Python module to show a progress meter for loops


In [1]:
! pip install -qU datasets sentence-transformers transformers numpy pandas tqdm

## Step 3: Download the evaluation dataset

We will use MongoDB's [cosmopedia-wikihow-chunked](https://huggingface.co/datasets/MongoDB/cosmopedia-wikihow-chunked) dataset, which has chunked versions of WikiHow articles from the [Cosmopedia](https://huggingface.co/datasets/HuggingFaceTB/cosmopedia) dataset released by Hugging Face. The dataset is pretty large, so we will only grab the first 25k records for testing.


In [2]:
from datasets import load_dataset
import pandas as pd

# Use streaming=True to load the dataset without downloading it fully
data = load_dataset("MongoDB/cosmopedia-wikihow-chunked", split="train", streaming=True)
# Get first 25k records from the dataset
data_head = data.take(25000)
df = pd.DataFrame(data_head)

# Use this if you want the full dataset
# data = load_dataset("AIatMongoDB/cosmopedia-wikihow-chunked", split="train")
# df = pd.DataFrame(data)

## Step 4: Data analysis

Make sure the length of the dataset is what we expect (25k), preview the data, drop Nones etc.


In [3]:
# Ensuring length of dataset is what we expect i.e. 25k
len(df)

25000

In [4]:
# Previewing the contents of the data
df.head()

,doc_id,chunk_id,text_token_length,text
0,0,0,180,Title: How to Create and Maintain a Compost Pi...
1,0,1,141,**Step 2: Gather Materials**\nGather brown (ca...
2,0,2,182,_Key guideline:_ For every volume of green mat...
3,0,3,188,_Key tip:_ Chop large items like branches and ...
4,0,4,157,**Step 7: Maturation and Use**\nAfter 3-4 mont...


In [5]:
# Only keep records where the text field is not null
df = df[df["text"].notna()]

In [6]:
# Number of unique documents in the dataset
df.doc_id.nunique()

4335

## Step 5: Creating embeddings

Define the embedding function, and run a quick test.


In [7]:
from typing import List
from transformers import AutoModel, AutoTokenizer
import torch

In [8]:
# Instruction to append to user queries, to improve retrieval
RETRIEVAL_INSTRUCT = "Represent this sentence for searching relevant passages:"

# Check if CUDA (GPU support) is available, and set the device accordingly
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# Load the UAE-Large-V1 model from the Hugging Face
model = AutoModel.from_pretrained("WhereIsAI/UAE-Large-V1").to(device)
# Load the tokenizer associated with the UAE-Large-V1 model
tokenizer = AutoTokenizer.from_pretrained("WhereIsAI/UAE-Large-V1")

In [9]:
# Decorator to disable gradient calculations
@torch.no_grad()
def get_embeddings(docs: List[str], input_type: str) -> List[List[float]]:
    """
    Get embeddings using the UAE-Large-V1 model.

    Args:
        docs (List[str]): List of texts to embed
        input_type (str): Type of input to embed. Can be "document" or "query".

    Returns:
        List[List[float]]: Array of embedddings
    """
    # Prepend retrieval instruction to queries
    if input_type == "query":
        docs = ["{}{}".format(RETRIEVAL_INSTRUCT, q) for q in docs]
    # Tokenize input texts
    inputs = tokenizer(
        docs, padding=True, truncation=True, return_tensors="pt", max_length=512
    ).to(device)
    # Pass tokenized inputs to the model, and obtain the last hidden state
    last_hidden_state = model(**inputs, return_dict=True).last_hidden_state
    # Extract embeddings from the last hidden state
    embeddings = last_hidden_state[:, 0]
    return embeddings.cpu().numpy()

In [10]:
test_angle_embed = get_embeddings([df.iloc[0]["text"]], "document")

In [11]:
len(test_angle_embed[0])

1024

## Step 6: Evaluation


### Measuring embedding latency

Create a local vector store (list) of embeddings for the entire dataset.


In [12]:
from tqdm.auto import tqdm

In [13]:
texts = df["text"].tolist()

In [14]:
batch_size = 128

In [15]:
embeddings = []
# Generate embeddings in batches
for i in tqdm(range(0, len(texts), batch_size)):
    end = min(len(texts), i + batch_size)
    batch = texts[i:end]
    # Generate embeddings for current batch
    batch_embeddings = get_embeddings(batch, "document")
    # Add to the list of embeddings
    embeddings.extend(batch_embeddings)

  0%|          | 0/196 [00:00<?, ?it/s]

### Measuring retrieval quality

- Create embedding for the user query
<p>
- Get the top 5 most similar documents from the local vector store using cosine similarity as the similarity metric


In [16]:
import numpy as np
from sentence_transformers.util import cos_sim

In [17]:
# Converting embeddings list to a Numpy array- required to calculate cosine similarity
embeddings = np.asarray(embeddings)

In [18]:
def query(query: str, top_k: int = 3) -> None:
    """
    Query the local vector store for the top 3 most relevant documents.

    Args:
        query (str): User query
        top_k (int, optional): Number of documents to return. Defaults to 3.
    """
    # Generate embedding for the user query
    query_emb = np.asarray(get_embeddings([query], "query"))
    # Calculate cosine similarity
    scores = cos_sim(query_emb, embeddings)[0]
    # Get indices of the top k records
    idxs = np.argsort(-scores)[:top_k]

    print(f"Query: {query}")
    for idx in idxs:
        print(f"Score: {scores[idx]:.4f}")
        print(texts[idx])
        print("--------")

In [19]:
query_emb = query("Give me some tips to improve my mental health.")

Query: Give me some tips to improve my mental health.
Score: 0.7515
Key Tips & Guidelines:

* Acknowledge that prioritizing mental health is not selfish but necessary for maintaining optimal functioning.
* Accept that everyone experiences ups and downs; seeking support when needed demonstrates strength rather than weakness.

Step 2: Practice Self-Awareness
Developing mindfulness skills allows individuals to become aware of their thoughts, feelings, and physical sensations without judgment. Being present in the moment fosters self-compassion, promotes relaxation, and enhances problem-solving abilities.

Key Tips & Guidelines:

* Set aside time each day for meditation or deep breathing exercises.
* Journal regularly to explore thoughts and emotions objectively.
* Engage in activities (e.g., yoga, tai chi) that encourage both physical movement and introspection.

Step 3: Establish Healthy Lifestyle Habits
Physical health significantly impacts mental well-being. Adopting healthy habits con

In [20]:
query_emb = query("Give me some tips for writing good code.")

Query: Give me some tips for writing good code.
Score: 0.7326
Step 6: Improve Code Quality
Strive for clean, readable, maintainable code. Adopt consistent naming conventions, indentation styles, and formatting rules. Utilize version control systems like Git to track changes and collaborate effectively. Leverage linters and static analyzers to enforce style guides automatically. Document your work using comments and dedicated documentation tools. High-quality code facilitates collaboration, promotes longevity, and simplifies troubleshooting.

Step 7: Embrace Best Practices
Follow established best practices relevant to your chosen language and domain. Examples include Object-Oriented Design Principles, SOLID principles, Test-Driven Development (TDD), Dependency Injection, Asynchronous Programming, etc. While seemingly overwhelming initially, integrating them gradually enhances design patterns, scalability, and extensibility. Consult authoritative blogs, books, and articles to stay update

In [21]:
query("How do I create a basic webpage?")

Query: How do I create a basic webpage?
Score: 0.8068
Creating a webpage with basic HTML is an exciting endeavor that allows you to share information, showcase your creativity, or build the foundation for a more complex website. This comprehensive guide will walk you through each step required to create a simple yet effective static webpage using HTML (HyperText Markup Language), the standard language used to structure content on the World Wide Web.

**Step 1: Choose a Text Editor**

Before diving into coding, select a suitable text editor for writing your HTML code. Popular choices include Notepad++ (Windows), Visual Studio Code (cross-platform), Atom (cross-platform), and Sublime Text (cross-platform). These editors offer features such as syntax highlighting, auto-completion, and error detection, making them ideal for beginners and professionals alike.

Key tip: Avoid using word processors like Microsoft Word or Google Docs, as they may add unnecessary formatting codes that can inter

In [22]:
query(
    "What are some environment-friendly practices I can incorporate in everyday life?"
)

Query: What are some environment-friendly practices I can incorporate in everyday life?
Score: 0.7612
Step 15: Share Knowledge With Others
Educate friends, family, and colleagues about eco-friendly practices and benefits. Offer support during their transitions. Key tip: Lead by example without judgment. Guideline: Host workshops or discussion groups centered around sustainable living topics.

Conclusion:
By following these 15 steps, you'll become an earthy girl who embodies respect for our planet and its resources. Remember, every effort counts—big or small! Celebrate successes along the journey while striving for continuous improvement.
--------
Score: 0.7339
* **Carry Reusable Bags**: Keep cloth or other reusable shopping bags handy for grocery trips and errands. Store extra bags in your car, purse, or backpack for convenience.
* **Choose Sustainable Alternatives**: Opt for reusable water bottles, coffee cups, straws, and cutlery made from eco-friendly materials like stainless steel,